<a href="https://colab.research.google.com/github/yucan1028/thesis_code/blob/main/scenario_new_facilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade matplotlib
!pip install japanize-matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 58.1 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.1
    Uninstalling matplotlib-3.7.1:


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import random
import matplotlib.pyplot as plt
import calendar as cal
from datetime import timedelta
from scipy.optimize import minimize
import matplotlib

In [ ]:
import pandas as pd
import matplotlib as plt
import japanize_matplotlib #日本語化matplotlib
import seaborn as sns
sns.set(font="IPAexGothic") #日本語フォント設定

In [ ]:
# 日本語フォントの設定
matplotlib.rcParams['font.family'] = 'IPAexGothic'

In [ ]:
import matplotlib.pyplot as plt

##短縮バージョン

##エージェント作成

##キャリブレーション

In [ ]:
# 新潟県の観光客数予測係数
coefficients = {
    1: (103.04, -202751.93),
    2: (-1.39, 7507.21),
    3: (324.50, -648962.86),
    4: (289.50, -577025.86),
    5: (122.25, -239674.93),
    6: (47.54, -90435.79),
    7: (3.36, 116.00),
    8: (43.54, -75081.79),
    9: (11.75, -17978.07),
    10: (66.96, -128122.64),
    11: (42.07, -79626.86),
    12: (52.64, -102553.14)
}

# 佐渡島の観光客数の平均比率
average_ratios = {
    1: 0.00526,
    2: 0.00341,
    3: 0.00773,
    4: 0.01297,
    5: 0.02704,
    6: 0.03784,
    7: 0.02975,
    8: 0.02776,
    9: 0.03061,
    10: 0.02391,
    11: 0.01547,
    12: 0.00663
}

# 実際の観光客数のデータ
actual_tourist_counts = {
    (2011, 1): 29, (2012, 1): 28, (2013, 1): 24, (2014, 1): 27, (2015, 1): 24, (2016, 1): 23, (2017, 1): 21, (2018, 1): 24,
    (2011, 2): 20, (2012, 2): 19, (2013, 2): 16, (2014, 2): 17, (2015, 2): 14, (2016, 2): 16, (2017, 2): 11, (2018, 2): 14,
    (2011, 3): 29, (2012, 3): 35, (2013, 3): 38, (2014, 3): 36, (2015, 3): 36, (2016, 3): 41, (2017, 3): 38, (2018, 3): 28,
    (2011, 4): 54, (2012, 4): 96, (2013, 4): 100, (2014, 4): 86, (2015, 4): 75, (2016, 4): 66, (2017, 4): 63, (2018, 4): 82,
    (2011, 5): 141, (2012, 5): 189, (2013, 5): 191, (2014, 5): 176, (2015, 5): 172, (2016, 5): 180, (2017, 5): 184, (2018, 5): 163,
    (2011, 6): 191, (2012, 6): 227, (2013, 6): 236, (2014, 6): 224, (2015, 6): 199, (2016, 6): 195, (2017, 6): 174, (2018, 6): 172,
    (2011, 7): 239, (2012, 7): 253, (2013, 7): 225, (2014, 7): 201, (2015, 7): 203, (2016, 7): 193, (2017, 7): 168, (2018, 7): 166,
    (2011, 8): 392, (2012, 8): 416, (2013, 8): 381, (2014, 8): 321, (2015, 8): 327, (2016, 8): 328, (2017, 8): 310, (2018, 8): 320,
    (2011, 9): 187, (2012, 9): 222, (2013, 9): 182, (2014, 9): 182, (2015, 9): 172, (2016, 9): 172, (2017, 9): 143, (2018, 9): 134,
    (2011, 10): 194, (2012, 10): 213, (2013, 10): 173, (2014, 10): 160, (2015, 10): 148, (2016, 10): 158, (2017, 10): 127, (2018, 10): 117,
    (2011, 11): 101, (2012, 11): 110, (2013, 11): 85, (2014, 11): 78, (2015, 11): 80, (2016, 11): 66, (2017, 11): 55, (2018, 11): 58,
    (2011, 12): 26, (2012, 12): 25, (2013, 12): 24, (2014, 12): 22, (2015, 12): 19, (2016, 12): 26, (2017, 12): 26, (2018, 12): 16,
}

# 年ごとの平均レビュースコアを格納する辞書
annual_review_averages = {
    2011: 2.9258606425670117,
    2012: 3.0164292657943963,
    2013: 3.110876348585283,
    2014: 3.2203552079844324,
    2015: 3.1497245451183953,
    2016: 3.1512590863985337,
    2017: 2.932871734209716,
    2018: 2.966819683388602
}


##デルタとラムダの最適化

In [ ]:
def predict_tourist_counts(year, month, delta, lambda_, review_averages):
    # 新潟県の観光客数を計算
    niigata_tourists = coefficients[month][0] * year + coefficients[month][1]

    # 佐渡島の潜在的な観光客数を計算
    sado_potential = niigata_tourists * delta

    # 佐渡島へ行く観光客数を計算
    previous_year = year - 1 if year > 2011 else 2011
    review_adjustment = (review_averages.get(previous_year, 2.5) - 2.5) * lambda_
    sado_tourists = sado_potential * (1 + review_adjustment)

    return int(round(sado_tourists))

def calibrate_parameters(years):
    best_params = {}

    for year in years:
        min_error = float('inf')
        best_delta = 0
        best_lambda = 0

        for delta in np.linspace(0, 0.1, 20):  # Δの範囲を調整
            for lambda_value in np.linspace(0, 0.1, 20):  # λの範囲を調整
                total_error = 0
                for month in range(1, 13):
                    predicted = predict_tourist_counts(year, month, delta, lambda_value, annual_review_averages)
                    actual = actual_tourist_counts.get((year, month), 0)
                    total_error += abs(predicted - actual)

                if total_error < min_error:
                    min_error = total_error
                    best_delta = delta
                    best_lambda = lambda_value

        best_params[year] = (best_delta, best_lambda)

    return best_params

# キャリブレーションの実行
calibrated_params = calibrate_parameters(range(2011, 2019))
for year, params in calibrated_params.items():
    print(f"Year {year}: Δ = {params[0]}, λ = {params[1]}")


In [ ]:
calibrated_params

##平均割合の最適化

##月ごとに平均割合（季節性）を最適化

In [ ]:
# 年ごとに最適化された月別平均割合を計算する関数
def optimize_monthly_ratios_for_year(year):
    delta, lambda_ = calibrated_params[year]
    optimized_ratios = []

    for month in range(1, 13):
        def calculate_error(monthly_ratio):
            niigata_tourists = coefficients[month][0] * year + coefficients[month][1]
            sado_potential = niigata_tourists * (monthly_ratio + delta)
            review_adjustment = (annual_review_averages.get(year-1, 2.5) - 2.5) * lambda_
            predicted = sado_potential * (1 + review_adjustment)
            actual = actual_tourist_counts.get((year, month), 0)
            return abs(predicted - actual)

        result = minimize(calculate_error, 0.05)
        optimized_ratios.append(result.x[0])

    return optimized_ratios

# 各年の最適化された月別平均割合を計算し、グラフをプロット
fig, axs = plt.subplots(8, 1, figsize=(10, 40))

for i, year in enumerate(range(2011, 2019)):
    optimized_ratios = optimize_monthly_ratios_for_year(year)
    predicted_counts = []
    actual_counts = [actual_tourist_counts.get((year, m), 0) for m in range(1, 13)]

    for month, ratio in zip(range(1, 13), optimized_ratios):
        niigata_tourists = coefficients[month][0] * year + coefficients[month][1]
        sado_potential = niigata_tourists * (ratio + calibrated_params[year][0])
        review_adjustment = (annual_review_averages.get(year-1, 2.5) - 2.5) * calibrated_params[year][1]
        predicted = int(round(sado_potential * (1 + review_adjustment)))
        predicted_counts.append(predicted)

    axs[i].plot(range(1, 13), predicted_counts, label='Predicted Counts')
    axs[i].plot(range(1, 13), actual_counts, label='Actual Counts', linestyle='--')
    axs[i].set_title(f'{year} Tourist Counts: Predicted vs Actual')
    axs[i].set_xlabel('Month')
    axs[i].set_ylabel('Tourist Counts')
    axs[i].legend()

plt.tight_layout()
plt.show()


In [ ]:
# 各年の最適化された月別平均割合を計算し、データフレームに格納
data = []
for year in range(2011, 2019):
    optimized_ratios = optimize_monthly_ratios_for_year(year)
    for month, ratio in enumerate(optimized_ratios, start=1):
        data.append({
            'Year': year,
            'Month': month,
            'Optimized Ratio': ratio
        })

# データフレームの作成
optimized_ratios_df = pd.DataFrame(data)

# データフレームの表示
optimized_ratios_df


In [ ]:
# 各年の最適化された月別平均割合を計算
all_years_optimized_ratios = {}
for year in range(2011, 2019):
    all_years_optimized_ratios[year] = optimize_monthly_ratios_for_year(year)

# 各月の全年平均の月別平均割合を算出
monthly_average_ratios = []
for month in range(1, 13):
    monthly_total = 0
    for year in range(2011, 2019):
        monthly_total += all_years_optimized_ratios[year][month - 1]
    monthly_average = monthly_total / 8  # 8年間の平均
    monthly_average_ratios.append(monthly_average)

# 結果の表示
print("Monthly Average Ratios:", monthly_average_ratios)


In [ ]:
# 月別平均割合
monthly_average_ratios = [-0.013378606954303655, -0.01468921242301886, -0.011726550812103453,
                          -0.008050605274971782, 0.001394634020823737, 0.009775035487700808,
                          0.00425331514222259, 0.0026015092432105356, 0.004595362944347498,
                          7.518520156755508e-05, -0.005902048809618914, -0.012402855629269537]

# 年別に予測値と実際の値をプロット
fig, axs = plt.subplots(8, 1, figsize=(10, 40))

for i, year in enumerate(range(2011, 2019)):
    delta, lambda_ = calibrated_params[year]
    predicted_counts = []
    actual_counts = [actual_tourist_counts.get((year, m), 0) for m in range(1, 13)]

    for month in range(1, 13):
        niigata_tourists = coefficients[month][0] * year + coefficients[month][1]
        sado_potential = niigata_tourists * (monthly_average_ratios[month - 1] + delta)
        review_adjustment = (annual_review_averages.get(year - 1, 2.5) - 2.5) * lambda_
        predicted = int(round(sado_potential * (1 + review_adjustment)))
        predicted_counts.append(predicted)

    axs[i].plot(range(1, 13), predicted_counts, label='Predicted Counts')
    axs[i].plot(range(1, 13), actual_counts, label='Actual Counts', linestyle='--')
    axs[i].set_title(f'{year} Tourist Counts: Predicted vs Actual')
    axs[i].set_xlabel('Month')
    axs[i].set_ylabel('Tourist Counts')
    axs[i].legend()

plt.tight_layout()
plt.show()


##平均λ、Δバージョン


In [ ]:
# 平均Δと平均λを使用して年ごとの予測値をプロットする関数
def plot_predicted_vs_actual_with_averages():
    fig, axs = plt.subplots(8, 1, figsize=(10, 40))

    for i, year in enumerate(range(2011, 2019)):
        predicted_counts = []
        actual_counts = [actual_tourist_counts.get((year, m), 0) for m in range(1, 13)]

        for month in range(1, 13):
            niigata_tourists = coefficients[month][0] * year + coefficients[month][1]
            sado_potential = niigata_tourists * (monthly_average_ratios[month - 1] + average_delta)
            review_adjustment = (annual_review_averages.get(year - 1, 2.5) - 2.5) * average_lambda
            predicted = int(round(sado_potential * (1 + review_adjustment)))
            predicted_counts.append(predicted)

        axs[i].plot(range(1, 13), predicted_counts, label='Predicted Counts')
        axs[i].plot(range(1, 13), actual_counts, label='Actual Counts', linestyle='--')
        axs[i].set_title(f'{year} Tourist Counts: Predicted vs Actual')
        axs[i].set_xlabel('Month')
        axs[i].set_ylabel('Tourist Counts')
        axs[i].legend()

    plt.tight_layout()
    plt.show()

# 月別平均割合
monthly_average_ratios = [-0.013378606954303655, -0.01468921242301886, -0.011726550812103453,
                          -0.008050605274971782, 0.001394634020823737, 0.009775035487700808,
                          0.00425331514222259, 0.0026015092432105356, 0.004595362944347498,
                          7.518520156755508e-05, -0.005902048809618914, -0.012402855629269537]

# 平均Δと平均λの計算
total_delta = sum(params[0] for params in calibrated_params.values())
total_lambda = sum(params[1] for params in calibrated_params.values())
average_delta = total_delta / len(calibrated_params)
average_lambda = total_lambda / len(calibrated_params)

# 予測と実際のデータをプロットする関数を修正して再実行
plot_predicted_vs_actual_with_averages()



In [ ]:
average_delta

In [ ]:
average_delta

##ヒートマップ

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

def predict_tourist_counts_with_ratio(year, month, delta, lambda_, monthly_ratio, review_averages, coefficients):
    # 新潟県の観光客数を計算
    niigata_tourists = coefficients[month][0] * year + coefficients[month][1]

    # 佐渡島の潜在的な観光客数を計算
    sado_potential = niigata_tourists * (monthly_ratio + delta)

    # 佐渡島へ行く観光客数を計算
    previous_year = year - 1 if year > 2011 else 2011
    review_adjustment = (review_averages.get(previous_year, 2.5) - 2.5) * lambda_
    sado_tourists = sado_potential * (1 + review_adjustment)

    return int(round(sado_tourists))

def calculate_errors_for_heatmap(year, delta, lambda_, monthly_average_ratios, review_averages, coefficients):
    total_error = 0
    for month in range(1, 13):
        predicted = predict_tourist_counts_with_ratio(year, month, delta, lambda_, monthly_average_ratios[month - 1], review_averages, coefficients)
        actual = actual_tourist_counts.get((year, month), 0)
        total_error += abs(predicted - actual)
    return total_error

# Δとλの範囲を設定（0以上に調整）
delta_range = np.linspace(0, 0.1, 20)
lambda_range = np.linspace(0, 2, 20)

# 年別に散布図を生成
fig, axs = plt.subplots(8, 1, figsize=(10, 40))

for i, year in enumerate(range(2011, 2019)):
    errors = []
    for delta in delta_range:
        for lambda_value in lambda_range:
            error = calculate_errors_for_heatmap(year, delta, lambda_value, monthly_average_ratios, annual_review_averages, coefficients)
            errors.append((delta, lambda_value, error))

    deltas, lambdas, total_errors = zip(*errors)
    scatter = axs[i].scatter(lambdas, deltas, c=total_errors, cmap='coolwarm', marker='o', edgecolor='k')

    # 最小誤差の座標を見つけ、表示
    min_error_idx = np.argmin(total_errors)
    min_delta, min_lambda, _ = errors[min_error_idx]
    axs[i].annotate(f'({min_lambda:.2f}, {min_delta:.2f})', (min_lambda, min_delta), textcoords="offset points", xytext=(5,5), ha='center', fontsize=8, color='blue')

    axs[i].set_title(f'Year {year}')
    axs[i].set_xlabel('λ')
    axs[i].set_ylabel('Δ')

# カラーバーを追加
cbar = fig.colorbar(scatter, ax=axs, orientation='horizontal', pad=0.1, fraction=0.015, aspect=40, shrink=0.5)
cbar.set_label('Total Error')
plt.tight_layout()
plt.show()


In [ ]:
# Δとλの範囲を設定
delta_range = np.linspace(0, 0.1, 20)
lambda_range = np.linspace(0, 2, 20)

# 各年の累積誤差を計算する関数
def calculate_total_errors_for_all_years(delta, lambda_):
    total_error_all_years = 0
    for year in range(2011, 2019):
        total_error_all_years += calculate_errors_for_heatmap(year, delta, lambda_, monthly_average_ratios, annual_review_averages, coefficients)
    return total_error_all_years

# ヒートマップ用のデータを生成
errors = []
for delta in delta_range:
    for lambda_value in lambda_range:
        error = calculate_total_errors_for_all_years(delta, lambda_value)
        errors.append((delta, lambda_value, error))

# Δ, λ, および累積誤差を抽出
deltas, lambdas, total_errors = zip(*errors)

# 最小誤差の点を見つける
min_error = min(total_errors)
min_error_idx = total_errors.index(min_error)
min_delta, min_lambda = deltas[min_error_idx], lambdas[min_error_idx]

# ヒートマップの作成
plt.figure(figsize=(10, 8))
scatter = plt.scatter(lambdas, deltas, c=total_errors, cmap='coolwarm', marker='o', edgecolor='k')

# 最小誤差の点に注釈を追加
plt.annotate(f'Min Error\nΔ={min_delta:.2f}, λ={min_lambda:.2f}',
             xy=(min_lambda, min_delta),
             xytext=(min_lambda + 0.1, min_delta + 0.02),
             arrowprops=dict(facecolor='black', shrink=0.05),
             fontsize=10, color='blue')

# カラーバーを追加
cbar = plt.colorbar(scatter, orientation='horizontal', pad=0.1, fraction=0.015, aspect=40, shrink=0.5)
cbar.set_label('Total Cumulative Error (2011-2018)')

# タイトルとラベルの設定
plt.title('Total Cumulative Error Heatmap (2011-2018)')
plt.xlabel('λ')
plt.ylabel('Δ')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

def predict_tourist_counts_with_ratio(year, month, delta, lambda_, monthly_ratio, review_averages, coefficients):
    # 新潟県の観光客数を計算
    niigata_tourists = coefficients[month][0] * year + coefficients[month][1]

    # 佐渡島の潜在的な観光客数を計算
    sado_potential = niigata_tourists * (monthly_ratio + delta)

    # 佐渡島へ行く観光客数を計算
    previous_year = year - 1 if year > 2011 else 2011
    review_adjustment = (review_averages.get(previous_year, 2.5) - 2.5) * lambda_
    sado_tourists = sado_potential * (1 + review_adjustment)

    return int(round(sado_tourists))

def calculate_errors_for_heatmap(year, delta, lambda_, monthly_average_ratios, review_averages, coefficients):
    total_error = 0
    for month in range(1, 13):
        predicted = predict_tourist_counts_with_ratio(year, month, delta, lambda_, monthly_average_ratios[month - 1], review_averages, coefficients)
        actual = actual_tourist_counts.get((year, month), 0)
        total_error += abs(predicted - actual)
    return total_error

# Δとλの範囲を設定（0以上に調整）
delta_range = np.linspace(0, 0.1, 20)
lambda_range = np.linspace(0, 2, 20)

# 年別に散布図を生成
fig, axs = plt.subplots(8, 1, figsize=(10, 40))

for i, year in enumerate(range(2011, 2019)):
    errors = []
    for delta in delta_range:
        for lambda_value in lambda_range:
            error = calculate_errors_for_heatmap(year, delta, lambda_value, monthly_average_ratios, annual_review_averages, coefficients)
            errors.append((delta, lambda_value, error))

    deltas, lambdas, total_errors = zip(*errors)
    scatter = axs[i].scatter(lambdas, deltas, c=total_errors, cmap='coolwarm', marker='o', edgecolor='k')

    # 最小誤差の座標を見つけ、表示
    min_error_idx = np.argmin(total_errors)
    min_delta, min_lambda, _ = errors[min_error_idx]
    axs[i].annotate(f'({min_lambda:.2f}, {min_delta:.2f})', (min_lambda, min_delta), textcoords="offset points", xytext=(5,5), ha='center', fontsize=8, color='blue')

    axs[i].set_title(f'Year {year}')
    axs[i].set_xlabel('λ')
    axs[i].set_ylabel('Δ')

# カラーバーを追加
cbar = fig.colorbar(scatter, ax=axs, orientation='horizontal', pad=0.1, fraction=0.015, aspect=40, shrink=0.5)
cbar.set_label('Total Error')
plt.tight_layout()
plt.show()


##観光客生成

In [ ]:
review_averages = {
    2011: 2.9258606425670117,
    2012: 3.0164292657943963,
    2013: 3.110876348585283,
    2014: 3.2203552079844324,
    2015: 3.1497245451183953,
    2016: 3.1512590863985337,
    2017: 2.932871734209716,
    2018: 2.966819683388602
}

calibrated_params = {
    2011: (0.021052631578947368, 0.9473684210526315),
    2012: (0.021052631578947368, 1.263157894736842),
    2013: (0.02631578947368421, 0.21052631578947367),
    2014: (0.015789473684210527, 0.9473684210526315),
    2015: (0.015789473684210527, 0.8421052631578947),
    2016: (0.015789473684210527, 0.9473684210526315),
    2017: (0.010526315789473684, 1.263157894736842),
    2018: (0.010526315789473684, 1.3684210526315788)
}

# 月別平均比率
monthly_average_ratios = [-0.013378606954303655, -0.01468921242301886, -0.011726550812103453,
                          -0.008050605274971782, 0.001394634020823737, 0.009775035487700808,
                          0.00425331514222259, 0.0026015092432105356, 0.004595362944347498,
                          7.518520156755508e-05, -0.005902048809618914, -0.012402855629269537]


In [ ]:
import pandas as pd

# 佐渡島への実際の観光客数を計算する関数
def calculate_sado_tourists(year, month, delta, lambda_, review_averages):
    niigata_tourists = coefficients[month][0] * year + coefficients[month][1]
    sado_potential = niigata_tourists * (monthly_average_ratios[month - 1] + delta)
    review_adjustment = (review_averages.get(year - 1, 2.5) - 2.5) * lambda_
    sado_tourists = int(round(sado_potential * (1 + review_adjustment)))
    return sado_tourists

# 予測された観光客数を計算する関数
def create_predicted_tourist_counts(years, average_ratios, coefficients, review_averages, calibrated_params, average_delta, average_lambda):
    predicted_counts = []

    for year in years:
        for month in range(1, 13):
            # 2012年から2018年まではcalibrated_paramsを使用し、2019年以降は平均値を使用
            if year <= 2018:
                delta, lambda_value = calibrated_params[year]
            else:
                delta, lambda_value = average_delta, average_lambda

            predicted = calculate_sado_tourists(year, month, delta, lambda_value, review_averages)
            predicted_counts.append([year, month, predicted])

    predicted_counts_df = pd.DataFrame(predicted_counts, columns=['Year', 'Month', 'Predicted_Tourists'])
    predicted_counts_df.set_index(['Year', 'Month'], inplace=True)
    return predicted_counts_df

# 平均Δと平均λの計算
total_delta = sum(params[0] for params in calibrated_params.values())
total_lambda = sum(params[1] for params in calibrated_params.values())
average_delta = total_delta / len(calibrated_params)
average_lambda = total_lambda / len(calibrated_params)

# 2012年から2019年までの予測された観光客数を計算
years = range(2012, 2026)
predicted_counts_df = create_predicted_tourist_counts(years, average_ratios, coefficients, review_averages, calibrated_params, average_delta, average_lambda)

# 予測された観光客数を出力
print(predicted_counts_df)


In [ ]:
predicted_counts_df

##データベース生成関数

In [ ]:
def generate_tourists(num_tourists, preference_std_dev, year, month):
    tourists = []
    start_date = pd.Timestamp(f'{year}-{month:02d}-01')
    end_date = pd.Timestamp(f'{year}-{month:02d}-{pd.Timestamp(start_date).days_in_month}')

    for i in range(num_tourists):
        budget = random.choices([1, 2, 3, 4], weights=[0.06, 0.36, 0.37, 0.21])[0]
        # Pref4を除外して、3つの好みを生成
        preference = [max(1, min(5, random.gauss(2.5, preference_std_dev))) for _ in range(3)]
        schedule_pref = random.choices([0, 1], weights=[0.6, 0.4])[0]
        stay_duration = random.choices([0, 1, 2, 3, 4], weights=[0.3, 0.3, 0.2, 0.1, 0.1])[0]

        random_days = random.randint(0, (end_date - start_date).days)
        initial_day = start_date + timedelta(days=random_days)

        tourists.append([
            i + 1, budget, *preference, schedule_pref, stay_duration,
            0, 0, 0, 0, initial_day, 0
        ])

    columns = [
        "ID", "Budget", "Pref1", "Pref2", "Pref3", "Schedule_pref",
        "Stay_duration", "Expectation", "Experience",
        "Satisfaction", "Review", "Initial_day", "select_plan"
    ]
    tourists = pd.DataFrame(tourists, columns=columns)

    return tourists


def generate_facilities():
    facilities = [
        ["大佐渡スカイライン", 0.3, 5, 2, 1],
        ["白雲台", 0.7, 5, 1, 1],
        ["大野亀", 0.7, 5, 1, 1],
        ["二ツ亀", 0.5, 5, 1, 3],
        ["平根崎", 0.8, 5, 5, 4],
        ["尖閣湾揚島遊園", 0.4, 5, 1, 2],
        ["玉堂窯", 0.4, 1, 1, 1],
        ["佐渡金山", 0.7, 5, 1, 2],
        ["北沢浮遊選鉱場", 0.5, 2, 1, 1],
        ["きらりうむ佐渡", 0.7, 2, 1, 1],
        ["めおと岩観光", 0.8, 5, 1, 1],
        ["妙宣寺", 0.5, 3, 1, 1],
        ["根本寺", 0.4, 3, 1, 1],
        ["トキの森公園", 0.4, 4, 1, 2],
        ["佐渡博物館", 0.7, 3, 1, 1],
        ["両津郷土博物館", 0.6, 3, 1, 1],
        ["真野御陵", 0.6, 4, 1, 1],
        ["長浜荘", 0.3, 3, 1, 1],
        ["佐渡歴史伝説館", 0.7, 2, 1, 1],
        ["佐渡西三川ゴールドパーク", 0.8, 3, 1, 2],
        ["尾畑酒造", 0.5, 1, 1, 1],
        ["蓮華峰寺", 0.6, 4, 1, 1],
        ["矢島・経島", 0.2, 4, 2, 4],
        ["力屋観光汽船", 0.2, 4, 2, 5],
        ["宿根木集落", 0.5, 3, 1, 1],
        ["小木民俗博物館", 0.6, 1, 1, 3],
        ["小木家（小木港）", 0.8, 2, 1, 1],
        ["北雪酒造", 0.2, 1, 1, 1]
    ]

    columns = ["Facility_name", "congestion", "Attr1", "Attr2", "Attr3"]
    facilities_df = pd.DataFrame(facilities, columns=columns)
    return facilities_df

def create_plan(plan_id, facilities, plan_name):
    # Attr1, Attr2, Attr3 の値を取得
    attributes = facilities[['Attr1', 'Attr2', 'Attr3']].values
    congestions = facilities['congestion'].values
    sights = facilities['Facility_name'].values.tolist()

    # 平均値を計算
    attributes_avg = np.mean(attributes, axis=0)
    congestions_avg = np.mean(congestions)

    # その他の属性設定
    price = random.choices([1, 2, 3, 4], [0.1, 0.3, 0.4, 0.2])[0]
    selected_count = 0
    initial_rating = 3
    days = random.choice([0, 1, 2, 3, 4])
    deterioration = random.uniform(3, 5)

    # 広告力の設定（佐渡金山を含む特定のプランに関する設定を削除）
    ad_power = random.uniform(0, 0.2)  # すべてのプランに対してランダムな広告力を設定

    return {
        "ID": plan_id,
        "Plan_name": plan_name,
        "Attr1": attributes_avg[0],
        "Attr2": attributes_avg[1],
        "Attr3": attributes_avg[2],
        "Price": price,
        "Initial_rating": initial_rating,
        "Ad_power": ad_power,
        "Selected_count": selected_count,
        "Congestion": congestions_avg,
        "Days": days,
        "Sights": ', '.join(sights),
        "Deterioration": deterioration
    }



def generate_plans(num_single, num_double, num_triple, facilities):
    plans = []
    plan_id = 1

    # 残りのプランを生成
    for _, fac in facilities.iterrows():
        if num_single > 0:
            plan = create_plan(plan_id, pd.DataFrame([fac]), f"Plan_{plan_id}")
            plans.append(plan)
            plan_id += 1
            num_single -= 1

    # ランダムに2施設を含むプランを生成
    for _ in range(num_double):
        selected_facilities = facilities.sample(2)
        plan = create_plan(plan_id, selected_facilities, f"Plan_{plan_id}")
        plans.append(plan)
        plan_id += 1

    # ランダムに3施設を含むプランを生成
    for _ in range(num_triple):
        selected_facilities = facilities.sample(3)
        plan = create_plan(plan_id, selected_facilities, f"Plan_{plan_id}")
        plans.append(plan)
        plan_id += 1

    return pd.DataFrame(plans)



def generate_crowd(year, num_days):
    # 平日と土日の混雑度を定義する
    weekday_congestion = np.random.uniform(low=0.3, high=0.7, size=(num_days,))
    weekend_congestion = weekday_congestion * 1.5  # 土日は混雑度を1.5倍にする

    # 繁忙期の混雑度を定義する
    busy_months = [1, 3, 7, 8, 12]
    month_congestion = np.ones((num_days,))

    for i in busy_months:
        days_in_month = cal.monthrange(year, i)[1]
        start_day = sum(cal.monthrange(year, m)[1] for m in range(1, i))
        end_day = min(start_day + days_in_month, num_days)  # 配列のサイズを超えないように調整
        month_congestion[start_day:end_day] *= 1.4

    # 混雑度を計算
    congestion = []
    dates = []

    for month in range(1, 13):
        for day in range(1, cal.monthrange(year, month)[1] + 1):
            date = f"{year}-{month:02d}-{day:02d}"
            dates.append(date)
            day_of_week = cal.weekday(year, month, day)
            if day_of_week in [5, 6]:  # 土日
                congestion_value = weekend_congestion[len(congestion) % num_days] * (1.4 if month in busy_months else 1)
            else:  # 平日
                congestion_value = weekday_congestion[len(congestion) % num_days] * (1.4 if month in busy_months else 1)
            congestion.append(min(congestion_value, 1.0))  # 混雑度の最大値を1に制限

    crowd_df = pd.DataFrame({"Date": dates, "Congestion": congestion})
    return crowd_df.sample(frac=1).reset_index(drop=True)  # ランダムにシャッフルしてから返す

def is_leap_year(year):
    return year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)

def adjust_ad_power(plans, ad_power_change):
    adjusted_plans = plans.copy()
    adjusted_plans['Ad_power'] = adjusted_plans['Ad_power'] + ad_power_change
    return adjusted_plans



In [ ]:
# 平均Δと平均λの計算
total_delta = sum(params[0] for params in calibrated_params.values())
total_lambda = sum(params[1] for params in calibrated_params.values())
average_delta = total_delta / len(calibrated_params)
average_lambda = total_lambda / len(calibrated_params)

# 予測された観光客数を計算（平均値を使用）
predicted_counts_df = create_predicted_tourist_counts(range(2012, 2019), average_ratios, coefficients, review_averages, calibrated_params, average_delta, average_lambda)

# 施設データベースの生成
facilities = generate_facilities()

# プランの生成
plans = generate_plans(28, 41, 41, facilities)


##シミュレーション関数

In [ ]:
def select_plans(selected_tourists, plans):
    for index, tourist in selected_tourists.iterrows():
        # 観光客の選好ベクトルを作成
        pref_vec = np.array([tourist['Pref1'], tourist['Pref2'], tourist['Pref3']])

        # plansのAttrベクトルを作成し、観光客の選好ベクトルとのコサイン類似度を計算
        plans_vec = plans[['Attr1', 'Attr2', 'Attr3']].values
        similarity = cosine_similarity(pref_vec.reshape(1, -1), plans_vec)

        # コサイン類似度が大きい順にplansを並べ替え
        plans['similarity'] = similarity.reshape(-1)
        sorted_plans = plans.sort_values(['similarity'], ascending=False)

        # 観光客のStay_durationとplansのDaysが一致するもののみを選択
        sorted_plans = sorted_plans[sorted_plans['Days'] == tourist['Stay_duration']]

        # 予算に近い順に並び替えのためのコピーを作成
        sorted_plans_copy = sorted_plans.copy()
        sorted_plans_copy.loc[:, 'budget_diff'] = abs(sorted_plans_copy['Price'] - tourist['Budget'])
        sorted_plans_copy = sorted_plans_copy.sort_values(['budget_diff'])

        # Initial_ratingが大きい順に並び替え
        sorted_plans_copy = sorted_plans_copy.sort_values(['Initial_rating'], ascending=False)

        # 0.2の確率でランダムに並び替え
        if np.random.random() < 0.2:
            sorted_plans_copy = sorted_plans_copy.sample(frac=1).reset_index(drop=True)

        # 最も上にあるプランを実行プランとして選択
        if not sorted_plans_copy.empty:
            selected_plan = sorted_plans_copy.iloc[0]
            plans.loc[plans['ID'] == selected_plan['ID'], 'Selected_count'] += 1
            selected_tourists.at[index, 'select_plan'] = selected_plan['ID']
        else:
            selected_tourists.at[index, 'select_plan'] = None

    return selected_tourists




# 観光客に期待値が初期値として10点与えられる
INITIAL_EXPECTATION = 10

def calculate_expectation(selected_tourists, plans):
    for i, row in selected_tourists.iterrows():
        plan_id = row['select_plan']

        # 選択されたプランが存在しない場合、このループの残りの処理をスキップ
        if plan_id is None or not (plans['ID'] == plan_id).any():
            continue

        selected_plan = plans.loc[plans['ID'] == plan_id].iloc[0]

        # ゼロ除算や異常な値がないかチェック
        if selected_plan['Selected_count'] == 0 or np.isinf(selected_plan['Initial_rating']):
            print(f"Warning: Plan ID {plan_id} has an initial rating of {selected_plan['Initial_rating']} with {selected_plan['Selected_count']} selections.")
            continue

        # selected_planに書かれているIDのプランを取り出す
        selected_plan = plans[plans['ID'] == row['select_plan']].iloc[0]

        # 観光客の選好ベクトルを作成
        pref_vector = np.array([row['Pref1'], row['Pref2'], row['Pref3']])

        # プランのAttrベクトルを作成し、観光客の選好ベクトルとのコサイン類似度を計算
        attr_vector = selected_plan[['Attr1', 'Attr2', 'Attr3']].to_numpy()
        similarity = cosine_similarity([pref_vector], [attr_vector])

        # 嗜好を計算し、データフレームに記入
        element1 = similarity[0][0] * 0.15
        selected_tourists.at[i, 'element1'] = element1

def calculate_expectation(selected_tourists, plans):
    for i, row in selected_tourists.iterrows():
        plan_id = row['select_plan']

        # 選択されたプランが存在しない場合、このループの残りの処理をスキップ
        if plan_id is None or not (plans['ID'] == plan_id).any():
            continue

        selected_plan = plans.loc[plans['ID'] == plan_id].iloc[0]

        # ゼロ除算や異常な値がないかチェック
        if selected_plan['Selected_count'] == 0 or np.isinf(selected_plan['Initial_rating']):
            print(f"Warning: Plan ID {plan_id} has an initial rating of {selected_plan['Initial_rating']} with {selected_plan['Selected_count']} selections.")
            continue

        # selected_planに書かれているIDのプランを取り出す
        selected_plan = plans[plans['ID'] == row['select_plan']].iloc[0]

        # 観光客の選好ベクトルを作成
        pref_vector = np.array([row['Pref1'], row['Pref2'], row['Pref3']])

        # プランのAttrベクトルを作成し、観光客の選好ベクトルとのコサイン類似度を計算
        attr_vector = selected_plan[['Attr1', 'Attr2', 'Attr3']].to_numpy()
        similarity = cosine_similarity([pref_vector], [attr_vector])

        # 嗜好を計算し、データフレームに記入
        element1 = similarity[0][0] * 0.15
        selected_tourists.at[i, 'element1'] = element1

        # 金額を基にした要素の計算（0.6点満点）
        element2 = (selected_plan['Price'] / 4) * 0.6
        selected_tourists.at[i, 'element2'] = element2

        # 評価点数を基にした要素の計算（0.6点満点）
        element3 = (selected_plan['Initial_rating'] / 5) * 0.6
        selected_tourists.at[i, 'element3'] = element3

        # Ad_powerをデータフレームに記入
        ad_power = selected_plan['Ad_power']
        selected_tourists.at[i, 'Ad_power'] = ad_power

        # 期待値を計算
        expectation = (element1 + element2 + element3 + ad_power) * INITIAL_EXPECTATION
        selected_tourists.at[i, 'Expectation'] = expectation

    return selected_tourists


def calculate_experience(selected_tourists, plans, Ship_congestion_df):
    for i, tourist in selected_tourists.iterrows():
        plan_id = tourist["select_plan"]

        # 選択されたプランが存在しない場合、このループの残りの処理をスキップ
        if plan_id is None or not (plans['ID'] == plan_id).any():
            continue

        selected_plan = plans.loc[plans["ID"] == plan_id].iloc[0]

        # 'deterioration' 値をプランデータから取得
        deterioration = selected_plan["Deterioration"]

        # 'congestion' 値を計算
        congestion = selected_plan["Congestion"]
        if congestion <= 0.5:
            congestion_value = 5
        elif 0.5 < congestion < 0.8:
            congestion_value = 4
        elif 0.8 <= congestion <= 1.0:
            congestion_value = 3
        elif 1.0 < congestion < 1.2:
            congestion_value = 2
        elif 1.2 <= congestion < 1.4:
            congestion_value = 1
        else:
            congestion_value = 0

        # 観光客の 'Initial_day' と 'Ship_congestion_df' の 'Date' が一致するか確認し、'traffic' 値を計算
        ship_congestion_row = Ship_congestion_df[Ship_congestion_df["Date"] == tourist["Initial_day"].strftime("%Y-%m-%d")]
        if not ship_congestion_row.empty:
            ship_congestion = ship_congestion_row["Congestion"].iloc[0]
            if 0.4 <= ship_congestion < 0.7:
                ship_value = 5
            elif 0.7 <= ship_congestion < 0.9:
                ship_value = 3
            elif 0.9 <= ship_congestion < 1:
                ship_value = 1
            else:
                ship_value = 0
        else:
            ship_value = 0  # または適切なデフォルト値を設定

        # 'Overall satisfaction' をランダム値で計算
        overall_satisfaction = np.random.uniform(2, 5)
        if np.random.random() < 0.2:
            overall_satisfaction = 1

        # 'Experience' を計算し、データフレームに割り当て
        experience = congestion_value + deterioration + ship_value + overall_satisfaction
        selected_tourists.at[i, 'congestion'] = congestion_value
        selected_tourists.at[i, 'deterioration'] = deterioration
        selected_tourists.at[i, 'traffic'] = ship_value
        selected_tourists.at[i, 'Other'] = overall_satisfaction
        selected_tourists.at[i, 'Experience'] = experience

    # 正しい変数を返す
    return selected_tourists

def calculate_satisfaction(row, plans):
    experience = row['Experience']
    expectation = row['Expectation']
    select_plan = row['select_plan']

    # 選択したプランの選択回数を取得
    selected_count = plans.loc[plans['ID'] == select_plan, 'Selected_count'].iloc[0]

    # 選択回数に基づいた調整係数を適用
    adjustment = np.log(selected_count + 1)

    # 満足度の計算
    satisfaction = (experience - expectation) * adjustment
    return satisfaction

# 満足度をレビューに変換
def scale_to_review(satisfaction, min_satisfaction, max_satisfaction):
    if max_satisfaction == min_satisfaction:  # 分母が0になるのを防ぐ
        return 1 if satisfaction >= 0 else 5  # 体験が期待を上回れば最高点、下回れば最低点
    scaled_satisfaction = 1 + (satisfaction - min_satisfaction) / (max_satisfaction - min_satisfaction) * 4
    review = round(scaled_satisfaction, 1)  # 小数点第一位で四捨五入
    return review

def update_plan_ratings(selected_plans, plans):
    for index, row in selected_plans.iterrows():
        plan_id = row['select_plan']
        review = row['Review']

        current_rating = plans.loc[plans['ID'] == plan_id, 'Initial_rating'].iloc[0]
        selected_count = plans.loc[plans['ID'] == plan_id, 'Selected_count'].iloc[0]

        # 選択回数が少ない場合の調整
        if selected_count < 5:  # 例として5回を閾値とする
            weight = 0.5  # 重み付けを半分にする
        else:
            weight = 1  # 通常の重み付け

        new_total_rating = (current_rating * selected_count * weight) + review
        new_rating = new_total_rating / ((selected_count + 1) * weight)

        plans.loc[plans['ID'] == plan_id, 'Initial_rating'] = new_rating
        plans.loc[plans['ID'] == plan_id, 'Selected_count'] = selected_count + 1

    return plans



In [ ]:
def run_simulation(predicted_counts_df, facilities, plans):
    simulation_results = {}

    for (year, month), row in predicted_counts_df.iterrows():
        num_tourists = row['Predicted_Tourists']
        crowd_df = generate_crowd(year, 365)

        tourists = generate_tourists(num_tourists, 1, year, month)

        selected_tourists = select_plans(tourists, plans)
        selected_tourists = calculate_expectation(selected_tourists, plans)

        # 混雑度を更新
        for plan_id in plans['ID']:
            selected_count = selected_tourists[selected_tourists['select_plan'] == plan_id].shape[0]
            congestion_value = min(selected_count / 10, 1)
            plans.loc[plans['ID'] == plan_id, 'Congestion'] = congestion_value

        selected_tourists = calculate_experience(selected_tourists, plans, crowd_df)

        # Satisfaction列の計算と追加
        for index, row in selected_tourists.iterrows():
            satisfaction = calculate_satisfaction(row, plans)
            selected_tourists.at[index, 'Satisfaction'] = satisfaction

        max_satisfaction = selected_tourists['Satisfaction'].max()
        min_satisfaction = selected_tourists['Satisfaction'].min()
        selected_tourists['Review'] = selected_tourists['Satisfaction'].apply(lambda s: scale_to_review(s, min_satisfaction, max_satisfaction))

        plans = update_plan_ratings(selected_tourists, plans)

        simulation_results[(year, month)] = selected_tourists

    return simulation_results, plans


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def select_plans(selected_tourists, plans):
    for index, tourist in selected_tourists.iterrows():
        # 観光客の選好ベクトルを作成
        pref_vec = np.array([tourist['Pref1'], tourist['Pref2'], tourist['Pref3']])

        # plansのAttrベクトルを作成し、観光客の選好ベクトルとのコサイン類似度を計算
        plans_vec = plans[['Attr1', 'Attr2', 'Attr3']].values
        similarity = cosine_similarity(pref_vec.reshape(1, -1), plans_vec)

        # コサイン類似度が高い順にplansを並び替える
        sorted_plans = plans.loc[(similarity > 0).flatten()]
        sorted_plans['similarity'] = similarity.flatten()
        sorted_plans = sorted_plans.sort_values(by='similarity', ascending=False)

        # 予算に近い順に並び替え
        sorted_plans['budget_diff'] = abs(sorted_plans['Price'] - tourist['Budget'])
        sorted_plans = sorted_plans.sort_values(by='budget_diff')

        # Initial_ratingが高い順に並び替え
        sorted_plans = sorted_plans.sort_values(by='Initial_rating', ascending=False)

        # 最も上にあるプランを実行プランとして選択
        if not sorted_plans.empty:
            selected_plan = sorted_plans.iloc[0]
            # plansのSelected_countに1を足す
            plans.loc[plans['ID'] == selected_plan['ID'], 'Selected_count'] += 1
            # selected_touristsのselect_planに選択されたプランのIDを入力
            selected_tourists.at[index, 'select_plan'] = selected_plan['ID']
        else:
            # 適切なプランが見つからない場合の処理を追加
            selected_tourists.at[index, 'select_plan'] = None

    return selected_tourists


In [ ]:
# シミュレーションを実行して結果を取得
simulation_results, updated_plans = run_simulation(predicted_counts_df, facilities, plans)

# プランの選択回数と評価を取得
selected_counts = updated_plans['Selected_count']
ratings = updated_plans['Initial_rating']

# 相関係数を計算
correlation_coefficient = np.corrcoef(selected_counts, ratings)[0, 1]

# 散布図を作成
plt.scatter(selected_counts, ratings)
plt.xlabel('Selected Count')
plt.ylabel('Rating')
plt.title(f'Correlation between Selected Count and Rating\nCorrelation Coefficient: {correlation_coefficient:.2f}')
plt.show()


In [ ]:
# Selected_countが多い順にplansをソート
sorted_plans = plans.sort_values('Selected_count', ascending=False)

# ソートされたデータのグラフで可視化（IDを使用）
plt.figure(figsize=(30, 6))
plt.bar(sorted_plans['ID'].astype(str), sorted_plans['Selected_count'])
plt.xlabel('Plan ID')
plt.ylabel('Selected Count')
plt.title('Selected Count of Each Plan')
plt.xticks(rotation=45)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# プランごとの日別選択人数の集計
plan_daily_counts = {}

for key, dataframe in simulation_results.items():
    for index, row in dataframe.iterrows():
        plan_id = row['select_plan']
        date = row['Initial_day']
        if plan_id not in plan_daily_counts:
            plan_daily_counts[plan_id] = {}
        if date not in plan_daily_counts[plan_id]:
            plan_daily_counts[plan_id][date] = 0
        plan_daily_counts[plan_id][date] += 1

# 日別データをデータフレームに変換
plan_daily_counts_df = {plan_id: pd.Series(data).sort_index().cumsum()
                        for plan_id, data in plan_daily_counts.items()}

# 折れ線グラフの描画
plt.figure(figsize=(12, 6))
for plan_id, counts in plan_daily_counts_df.items():
    plt.plot(counts.index, counts, label=f'Plan {plan_id}')

plt.xlabel('Date')
plt.ylabel('Cumulative Number of Selections')
plt.title('Cumulative Number of Selections per Plan Over Time')
plt.legend()
plt.show()


In [ ]:
def generate_new_facilities(attributes, congestions):
    new_facilities = []
    for attr in attributes:
        for con in congestions:
            facility_name = f"新施設_{attr[0]}_{attr[1]}_{attr[2]}_混雑度{con}"
            new_facilities.append([facility_name, con, *attr])
    return pd.DataFrame(new_facilities, columns=["Facility_name", "congestion", "Attr1", "Attr2", "Attr3"])


In [ ]:
def generate_new_plans(new_facilities, facilities, plans):
    new_plans = []
    plan_id = max(plans['ID']) + 1

    for _, new_fac in new_facilities.iterrows():
        # 新しいプランにユニークなプレフィックスを追加
        new_plan_name = f"New_Plan_{plan_id}"
        plan = create_plan(plan_id, new_fac, new_plan_name)
        new_plans.append(plan)
        plan_id += 1

        existing_fac = facilities.sample()
        combined_facilities = pd.concat([pd.DataFrame([new_fac]), pd.DataFrame([existing_fac.iloc[0]])])
        new_plan_name = f"New_Plan_{plan_id}"
        plan = create_plan(plan_id, combined_facilities, new_plan_name)
        new_plans.append(plan)
        plan_id += 1

    return pd.DataFrame(new_plans)

def create_plan(plan_id, facilities, plan_name):
    # facilities が DataFrame でない場合、DataFrame に変換
    if not isinstance(facilities, pd.DataFrame):
        facilities = pd.DataFrame([facilities])

    attributes = []
    congestions = []
    sights = []

    for _, fac in facilities.iterrows():
        attributes.append([fac["Attr1"], fac["Attr2"], fac["Attr3"]])
        congestions.append(fac["congestion"])
        sights.append(fac["Facility_name"])

    attributes_avg = np.mean(attributes, axis=0) if attributes else [0, 0, 0]
    congestions_avg = np.mean(congestions) if congestions else 0

    price = random.choices([1, 2, 3, 4], [0.1, 0.3, 0.4, 0.2])[0]
    ad_power = random.uniform(0, 0.5)
    selected_count = 0
    initial_rating = 3
    days = random.choice([0, 1, 2, 3, 4])
    deterioration = random.uniform(3, 5)

    return {
        "ID": plan_id,
        "Plan_name": plan_name,
        "Attr1": attributes_avg[0],
        "Attr2": attributes_avg[1],
        "Attr3": attributes_avg[2],
        "Price": price,
        "Initial_rating": initial_rating,
        "Ad_power": ad_power,
        "Selected_count": selected_count,
        "Congestion": congestions_avg,
        "Days": days,
        "Sights": ', '.join(sights),
        "Deterioration": deterioration
    }


In [ ]:
def run_simulation_for_each_pattern(plans, facilities, attributes, congestions, predicted_counts_df):
    simulation_results = {}

    for attr in attributes:
        for con in congestions:
            # 新しい施設の生成
            new_facilities = generate_new_facilities([attr], [con])

            # 新しいプランの生成
            new_plans_df = generate_new_plans(new_facilities, facilities, plans)

            # 既存のプランに新プランを追加
            combined_plans = pd.concat([plans, new_plans_df])

            # シミュレーション実行
            result, updated_plans = run_simulation(predicted_counts_df, facilities, combined_plans)

            # 結果を保存
            simulation_results[(attr, con)] = result

    return simulation_results, updated_plans


In [ ]:
def select_plans(selected_tourists, plans):
    for index, tourist in selected_tourists.iterrows():
        # 観光客の選好ベクトルを作成
        pref_vec = np.array([tourist['Pref1'], tourist['Pref2'], tourist['Pref3']])

        # plansのAttrベクトルを作成し、観光客の選好ベクトルとのコサイン類似度を計算
        plans_vec = plans[['Attr1', 'Attr2', 'Attr3']].values
        similarity = cosine_similarity(pref_vec.reshape(1, -1), plans_vec)

        # コサイン類似度が大きい順にplansを並べ替え
        plans['similarity'] = similarity.reshape(-1)
        sorted_plans = plans.sort_values(['similarity'], ascending=False)

        # 観光客のStay_durationとplansのDaysが一致するもののみを選択
        sorted_plans = sorted_plans[sorted_plans['Days'] == tourist['Stay_duration']]

        # 予算に近い順に並び替えのためのコピーを作成
        sorted_plans_copy = sorted_plans.copy()
        sorted_plans_copy.loc[:, 'budget_diff'] = abs(sorted_plans_copy['Price'] - tourist['Budget'])
        sorted_plans_copy = sorted_plans_copy.sort_values(['budget_diff'])

        # Initial_ratingが大きい順に並び替え
        sorted_plans_copy = sorted_plans_copy.sort_values(['Initial_rating'], ascending=False)

        # 0.05の確率でランダムに並び替え
        if np.random.random() < 0.05:
            sorted_plans_copy = sorted_plans_copy.sample(frac=1).reset_index(drop=True)

        # 最も上にあるプランを実行プランとして選択
        if not sorted_plans_copy.empty:
            selected_plan = sorted_plans_copy.iloc[0]
            # plansのSelected_countに1を足す
            plans.loc[plans['ID'] == selected_plan['ID'], 'Selected_count'] += 1
            # selected_touristsのselect_planに選択されたプランのIDを入力
            selected_tourists.at[index, 'select_plan'] = selected_plan['ID']
        else:
            # デバッグ情報の出力
            print(f"No available plans for tourist ID {index}. Plans may be empty or no match found.")
            # ここに適切な代替処理を追加
            continue

    return selected_tourists


In [ ]:
def run_simulation_for_year(year, facilities, plans, coefficients, monthly_average_ratios, review_averages, calibrated_params, average_delta, average_lambda):
    simulation_results = {}

    for month in range(1, 13):
        # 予測された観光客数を計算
        if year <= 2018:
            delta, lambda_value = calibrated_params[year]
        else:
            delta, lambda_value = average_delta, average_lambda
        predicted_tourists = calculate_sado_tourists(year, month, delta, lambda_value, review_averages)

        # 混雑度データの生成
        crowd_df = generate_crowd(year, 365)

        # 観光客データを生成
        tourists = generate_tourists(predicted_tourists, preference_std_dev=1, year=year, month=month)
        selected_tourists = select_plans(tourists, plans)

        # 期待値、経験値、満足度の計算
        selected_tourists = calculate_expectation(selected_tourists, plans)
        selected_tourists = calculate_experience(selected_tourists, plans, crowd_df)
        for index, row in selected_tourists.iterrows():
            satisfaction = calculate_satisfaction(row, plans)
            selected_tourists.at[index, 'Satisfaction'] = satisfaction

        # レビューの計算
        max_satisfaction = selected_tourists['Satisfaction'].max()
        min_satisfaction = selected_tourists['Satisfaction'].min()
        selected_tourists['Review'] = selected_tourists['Satisfaction'].apply(lambda s: scale_to_review(s, min_satisfaction, max_satisfaction))

        # プランの評価を更新
        plans = update_plan_ratings(selected_tourists, plans)
        simulation_results[month] = selected_tourists

    return simulation_results

# シミュレーションの実行
facilities = generate_facilities()
for year in range(2024, 2027):
    # ここで generate_plans 関数を正しく呼び出す
    plans = generate_plans(28, 41, 41, facilities)  # num_single, num_double, num_triple, facilities の引数を渡す
    simulation_results = run_simulation_for_year(year, facilities, plans, coefficients, monthly_average_ratios, review_averages, calibrated_params, average_delta, average_lambda)
    # 結果の出力（必要に応じて）
    print(f"Results for {year}:")
    for month in simulation_results:
        print(f"Month: {month}, Results: {simulation_results[month].head()}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# プランの選択回数と評価を取得
selected_counts = plans['Selected_count']
ratings = plans['Initial_rating']

# 相関係数を計算
correlation_coefficient = np.corrcoef(selected_counts, ratings)[0, 1]

# 散布図を作成
plt.scatter(selected_counts, ratings)
plt.xlabel('Selected Count')
plt.ylabel('Rating')
plt.title(f'Correlation between Selected Count and Rating\nCorrelation Coefficient: {correlation_coefficient:.2f}')
plt.show()


In [ ]:
# Selected_countが多い順にplansをソート
sorted_plans = plans.sort_values('Selected_count', ascending=False)

# ソートされたデータのグラフで可視化（IDを使用）
plt.figure(figsize=(30, 6))
plt.bar(sorted_plans['ID'].astype(str), sorted_plans['Selected_count'])
plt.xlabel('Plan ID')
plt.ylabel('Selected Count')
plt.title('Selected Count of Each Plan')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# 2012年から2027年までの予測された観光客数を計算
years = range(2012, 2028)
predicted_counts_df = create_predicted_tourist_counts(years, average_ratios, coefficients, review_averages, calibrated_params, average_delta, average_lambda)


In [ ]:
# 新しい施設の特徴ベクトル
attributes = [(5, 2, 2), (2, 5, 2), (2, 2, 5), (3, 3, 3)]

# 混雑度のオプション
congestions = [0.2, 0.5, 0.7]

# 既存のプランと施設のデータをコピー
original_plans = plans.copy()
original_facilities = facilities.copy()

# 新しい施設のプランを生成し、既存のプランリストに追加
plan_id = len(original_plans) + 1  # 新しいプランIDの開始点
for attr in attributes:
    for con in congestions:
        # 新しい施設のデータフレームを作成
        new_facility_df = pd.DataFrame([[f"新しい施設_{attr}_{con}", con, attr[0], attr[1], attr[2]]],
                                       columns=["Facility_name", "congestion", "Attr1", "Attr2", "Attr3"])
        # 新しい施設を含むプランを生成 (仮にcreate_planが3つの引数しか取らない場合)
        new_plan = create_plan(plan_id, new_facility_df, f"Plan_{plan_id}")
        original_plans = pd.concat([original_plans, pd.DataFrame([new_plan])]).reset_index(drop=True)
        plan_id += 1


In [ ]:
def add_new_facility_plan(attribute, congestion, original_plans):
    # 新しいプランIDは111に固定
    plan_id = 111
    # 新しい施設のデータを作成
    new_facility = pd.DataFrame([[f"新しい施設_{attribute[0]}_{attribute[1]}_{attribute[2]}_混雑度{congestion}", congestion, *attribute]],
                                columns=["Facility_name", "congestion", "Attr1", "Attr2", "Attr3"])
    # 新しい施設を含むプランを生成
    new_plan = create_plan(plan_id, new_facility, f"Plan_{plan_id}")
    # 新しいプランを元のプランリストに追加
    return pd.concat([original_plans, pd.DataFrame([new_plan])]).reset_index(drop=True)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# 既存のプランと施設のデータをコピー
original_plans = plans.copy()
original_facilities = facilities.copy()

# 新しい施設の特徴ベクトルと混雑度
attributes = [(5, 2, 2), (2, 5, 2), (2, 2, 5), (3, 3, 3)]
congestions = [0.2, 0.5, 0.7]

# 各シナリオごとにシミュレーションを実行
for attr in attributes:
    for con in congestions:
        # 元のプランリストでシミュレーション実行（プラン110まで）
        results_without_new_facility = run_simulation_for_year(2024, facilities, original_plans, coefficients, monthly_average_ratios, review_averages, calibrated_params, average_delta, average_lambda)

        # 新しい施設のプランを追加
        combined_plans_with_new_facility = add_new_facility_plan(attr, con, original_plans)

        # 新しい施設を含むプランリストでシミュレーション実行
        results_with_new_facility = run_simulation_for_year(2024, facilities, combined_plans_with_new_facility, coefficients, monthly_average_ratios, review_averages, calibrated_params, average_delta, average_lambda)

        # 結果の集計（新しい施設を含む）
        selection_counts_with_new_facility = pd.Series(dtype=int)
        for month, monthly_results in results_with_new_facility.items():
            monthly_counts = monthly_results['select_plan'].value_counts()
            selection_counts_with_new_facility = selection_counts_with_new_facility.add(monthly_counts, fill_value=0)

        # 結果の集計（新しい施設を含まない）
        selection_counts_without_new_facility = pd.Series(dtype=int)
        for month, monthly_results in results_without_new_facility.items():
            monthly_counts = monthly_results['select_plan'].value_counts()
            selection_counts_without_new_facility = selection_counts_without_new_facility.add(monthly_counts, fill_value=0)

        # 新しい施設を含むバージョンの選択回数でソート
        sorted_indices = selection_counts_with_new_facility.sort_values(ascending=False).index

        # グラフの描画
        plt.figure(figsize=(20, 6))
        bar_width = 0.35
        index = np.arange(len(sorted_indices))

        bars1 = plt.bar(index - bar_width/2, selection_counts_without_new_facility.reindex(sorted_indices, fill_value=0), bar_width, label='Without New Facility')
        bars2 = plt.bar(index + bar_width/2, selection_counts_with_new_facility.reindex(sorted_indices, fill_value=0), bar_width, label='With New Facility', color='red')

        plt.xlabel('Plan ID')
        plt.ylabel('Total Selections')
        plt.title(f'Total Plan Selections for Attribute: {attr}, Congestion: {con}')
        plt.xticks(index, sorted_indices)
        plt.legend()
        plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 既存のプランと施設のデータをコピー
original_plans = plans.copy()
original_facilities = facilities.copy()

# 新しい施設の特徴ベクトルと混雑度
attributes = [(5, 2, 2), (2, 5, 2), (2, 2, 5), (3, 3, 3)]
congestions = [0.2, 0.5, 0.7]

# 各シナリオごとにシミュレーションを実行
for attr in attributes:
    for con in congestions:
        # 元のプランリストに戻す
        plans_reset = original_plans.copy()

        # 新しい施設のプランを追加
        combined_plans = add_new_facility_plan(attr, con, plans_reset)

        # シミュレーション実行
        yearly_results = run_simulation_for_year(2024, facilities, combined_plans, coefficients, monthly_average_ratios, review_averages, calibrated_params, average_delta, average_lambda)

        # 結果の集計
        yearly_selection_counts = pd.Series(dtype=int)
        for month, monthly_results in yearly_results.items():
            monthly_counts = monthly_results['select_plan'].value_counts()
            yearly_selection_counts = yearly_selection_counts.add(monthly_counts, fill_value=0)

        # プラン111が選択されなかった場合でも、0を割り当てる
        if 111 not in yearly_selection_counts:
            yearly_selection_counts[111] = 0

        # 選択回数の多い順にソート
        sorted_counts = yearly_selection_counts.sort_values(ascending=False)

        # グラフの描画
        plt.figure(figsize=(20, 6))
        for plan_id in sorted_counts.index:
            color = 'red' if plan_id == 111 else 'blue'  # プラン111は赤、他は青
            plt.bar(str(plan_id), sorted_counts[plan_id], color=color)
        plt.xlabel('Plan ID')
        plt.ylabel('Total Selections')
        plt.title(f'Total Plan Selections for Attribute: {attr}, Congestion: {con}')
        plt.xticks(rotation=90)
        plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 結果の可視化
for year, year_results in simulation_results.items():
    # 年間を通じてのプランの選択回数を集計
    yearly_selection_counts = pd.Series(dtype=int)

    for month, monthly_results in year_results.items():
        monthly_counts = monthly_results['select_plan'].value_counts()
        yearly_selection_counts = yearly_selection_counts.add(monthly_counts, fill_value=0)

    # 年間の選択回数を表示
    print(f"Year {year} - Plan Selection Counts:")
    print(yearly_selection_counts)

    # グラフの描画
    plt.figure(figsize=(12, 6))
    yearly_selection_counts.plot(kind='bar', color='skyblue')
    plt.xlabel('Plan ID')
    plt.ylabel('Total Selections')
    plt.title(f'Total Plan Selections in {year}')
    plt.xticks(rotation=45)
    plt.show()


In [ ]:
def calculate_cumulative_tourists_for_specific_plans(simulation_results, plan_ids):
    cumulative_tourists = {pattern: {plan_id: 0 for plan_id in plan_ids} for pattern in simulation_results.keys()}
    for pattern, month_data in simulation_results.items():
        for _, dataframe in month_data.items():
            for plan_id in plan_ids:
                tourists_count = dataframe[dataframe['select_plan'] == plan_id].shape[0]
                cumulative_tourists[pattern][plan_id] += tourists_count
    return cumulative_tourists

# 特定のプランIDを指定
specific_plan_ids = [26, 27]

# 累積観光客数の計算
cumulative_tourists_specific_plans = calculate_cumulative_tourists_for_specific_plans(simulation_results, specific_plan_ids)

# 各シナリオの累積観光客数の出力
for pattern, counts in cumulative_tourists_specific_plans.items():
    print(f"Pattern: {pattern}")
    for plan_id, total_tourists in counts.items():
        print(f"Plan {plan_id}: Cumulative Tourists: {total_tourists}")
    print()


In [ ]:
# 特定の施設を含むプランを選択した観光客数の累計を計算する関数
def calculate_cumulative_tourists_for_facilities(simulation_results, facilities, attributes, congestions):
    # 各パターンごとに累積値を格納する辞書
    cumulative_tourists_by_facility = {(attr, con): {facility_id: 0 for facility_id in facilities} for attr in attributes for con in congestions}

    # 各パターンについてシミュレーション結果を処理
    for (attr, con), results_by_month in simulation_results.items():
        for month_data in results_by_month.values():
            for facility_id in facilities:
                # 特定の施設IDを含むプランを選択した人数をカウント
                tourists_selected = month_data[month_data['select_plan'] == facility_id].shape[0]
                cumulative_tourists_by_facility[(attr, con)][facility_id] += tourists_selected

    return cumulative_tourists_by_facility

# 施設のIDのリストを作成
facility_ids = [facility['ID'] for facility in facilities]

# 累積観光客数を計算
cumulative_tourists_by_facility = calculate_cumulative_tourists_for_facilities(simulation_results, facility_ids, attributes, congestions)

# 累積観光客数を出力
for (attr, con), cumulative_counts in cumulative_tourists_by_facility.items():
    print(f"Attribute: {attr}, Congestion: {con}")
    for facility_id, count in cumulative_counts.items():
        print(f"Facility ID {facility_id}: {count}")
    print()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 各プランの累積選択数をプロットする関数
def plot_cumulative_selections(simulation_results, title):
    cumulative_selections = pd.DataFrame()

    for (year, month), tourists in simulation_results.items():
        monthly_selections = tourists['select_plan'].value_counts()
        if cumulative_selections.empty:
            cumulative_selections = monthly_selections
        else:
            cumulative_selections = cumulative_selections.add(monthly_selections, fill_value=0)

    cumulative_selections.sort_values(ascending=False).cumsum().plot(kind='line')
    plt.title(title)
    plt.xlabel('プランID')
    plt.ylabel('累積選択数')
    plt.show()

# シミュレーションを実行し、結果をプロット
for year in range(2024, 2027):
    facilities = generate_facilities()
    plans = generate_plans(25, facilities)
    simulation_results = run_simulation_for_year(year, facilities, plans, coefficients, monthly_average_ratios, review_averages, calibrated_params, average_delta, average_lambda)

    # 各年の累積選択数をプロット
    plot_cumulative_selections(simulation_results, f"{year}年の累積選択数")


In [ ]:
import pandas as pd

def run_simulation_multiple_times(years, facilities, attributes, congestions, num_iterations=10):
    all_simulation_results = []

    for _ in range(num_iterations):
        simulation_results = run_simulation_for_each_pattern(plans, facilities, attributes, congestions, create_predicted_tourist_counts(years, average_ratios, coefficients, review_averages, calibrated_params, average_delta, average_lambda))
        all_simulation_results.append(simulation_results)

    return all_simulation_results

def calculate_cumulative_selections(simulation_results):
    cumulative_selections = {}
    for key, data in simulation_results.items():
        plan_selections = {}
        for month_data in data.values():
            monthly_selections = month_data['select_plan'].value_counts()
            for plan_id, count in monthly_selections.items():
                plan_selections[plan_id] = plan_selections.get(plan_id, 0) + count
        cumulative_selections[key] = plan_selections
    return cumulative_selections

# シミュレーションを10回実行
all_results = run_simulation_multiple_times(range(2024, 2027), facilities, attributes, congestions, num_iterations=10)

# 各シミュレーションの累積選択数を計算
cumulative_selections_all = [calculate_cumulative_selections(results) for results in all_results]

# 累積選択数を集計
cumulative_selections_summary = {}
for i, cumulative_selections in enumerate(cumulative_selections_all):
    for key, selections in cumulative_selections.items():
        if key not in cumulative_selections_summary:
            cumulative_selections_summary[key] = []
        cumulative_selections_summary[key].append(sum(selections.values()))

# 結果の集計
for key, selections_list in cumulative_selections_summary.items():
    print(f"Key: {key}")
    print(f"Avg Cumulative Selections: {sum(selections_list) / len(selections_list)}")
    print(f"Min: {min(selections_list)}, Max: {max(selections_list)}")


In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import zipfile

# 保存するディレクトリを指定（存在しない場合は作成）
output_dir = 'graphs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 各特徴ベクトルと混雑度の組み合わせに対してグラフを作成
for attr in attributes:
    for con in congestions:
        key = (attr, con)
        if key in simulation_results:
            # すべての月のデータを結合
            all_months_df = pd.concat(simulation_results[key].values())

            # ファイル名の一部として使用するために特徴ベクトルと混雑度を文字列に変換
            attr_str = '_'.join(map(str, attr))
            con_str = str(int(con * 100)).zfill(2)

            # 選択回数の棒グラフ
            plan_counts = all_months_df['select_plan'].value_counts()
            plt.figure(figsize=(10, 6))
            plt.bar(plan_counts.index.astype(str), plan_counts.values)
            plt.xlabel('Plan ID')
            plt.ylabel('Selected Count')
            plt.title(f'Selected Count of Each Plan for Attribute: {attr}, Congestion: {con}')
            bar_graph_filename = f'{attr_str}_{con_str}_bar.png'
            plt.savefig(os.path.join(output_dir, bar_graph_filename))
            plt.close()

            # 選択回数とレビュー評価の散布図
            selected_counts = all_months_df['select_plan'].value_counts()
            reviews = all_months_df.groupby('select_plan')['Review'].mean()
            correlation_coefficient = np.corrcoef(selected_counts, reviews.reindex(selected_counts.index))[0, 1]

            plt.figure(figsize=(10, 6))
            plt.scatter(selected_counts.index, reviews.reindex(selected_counts.index))
            plt.xlabel('Selected Count')
            plt.ylabel('Average Review Rating')
            plt.title(f'Correlation between Selected Count and Review Rating for Attribute: {attr}, Congestion: {con}\nCorrelation Coefficient: {correlation_coefficient:.2f}')
            scatter_graph_filename = f'{attr_str}_{con_str}_scatter.png'
            plt.savefig(os.path.join(output_dir, scatter_graph_filename))
            plt.close()
        else:
            print(f"No results found for Attribute: {attr}, Congestion: {con}")

# ZIPファイルの名前を指定
zip_filename = 'graphs.zip'

# 生成したグラフの画像をZIPファイルに圧縮
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk(output_dir):
        for file in files:
            if file.endswith(".png"):
                zipf.write(os.path.join(root, file), file)


In [ ]:
def plot_selection_bar_chart(simulation_data, attribute, congestion):
    plan_counts = pd.Series(dtype=int)
    for month_data in simulation_data.values():
        monthly_counts = month_data['select_plan'].value_counts()
        plan_counts = plan_counts.add(monthly_counts, fill_value=0)

    plt.figure(figsize=(12, 6))
    plt.bar(plan_counts.index.astype(str), plan_counts.values)
    plt.xlabel('Plan ID')
    plt.ylabel('Total Selections')
    plt.title(f'Total Selections for Attribute: {attribute}, Congestion: {congestion}')
    plt.show()

def plot_cumulative_selections(simulation_data, attribute, congestion):
    cumulative_counts = pd.Series(dtype=int)
    for month_data in simulation_data.values():
        daily_counts = month_data.groupby('Initial_day')['select_plan'].count()
        cumulative_counts = cumulative_counts.add(daily_counts.cumsum(), fill_value=0)

    plt.figure(figsize=(12, 6))
    plt.plot(cumulative_counts.index, cumulative_counts.values)
    plt.xlabel('Date')
    plt.ylabel('Cumulative Selections')
    plt.title(f'Cumulative Selections Over Time for Attribute: {attribute}, Congestion: {congestion}')
    plt.show()

# 分析と比較のためのデータ集計
impact_analysis = {}

for attr in attributes:
    for con in congestions:
        key = (attr, con)
        if key in simulation_results:
            total_selections = sum(month_data['select_plan'].value_counts().sum() for month_data in simulation_results[key].values())
            impact_analysis[key] = total_selections

            # 選択回数の棒グラフ
            plot_selection_bar_chart(simulation_results[key], attr, con)

            # 累積選択人数の折れ線グラフ
            plot_cumulative_selections(simulation_results[key], attr, con)

# 最も影響が大きいシナリオの特定
most_impactful_scenario = max(impact_analysis, key=impact_analysis.get)
print(f"The most impactful scenario is with attribute: {most_impactful_scenario[0]} and congestion: {most_impactful_scenario[1]}")
print(f"Total selections for this scenario: {impact_analysis[most_impactful_scenario]}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 各シナリオの選択回数と累積選択人数の差分計算
selection_diffs = {}
cumulative_diffs = {}

for key, month_data in simulation_results.items():
    # 各月の選択回数集計
    total_counts = pd.Series(dtype=int)
    cumulative_counts = pd.Series(dtype=int)

    for monthly_data in month_data.values():
        monthly_counts = monthly_data['select_plan'].value_counts()
        total_counts = total_counts.add(monthly_counts, fill_value=0)

        daily_counts = monthly_data.groupby('Initial_day')['select_plan'].count().cumsum()
        cumulative_counts = cumulative_counts.add(daily_counts, fill_value=0)

    # 選択回数の差分
    selection_diffs[key] = total_counts
    # 累積選択人数の差分
    cumulative_diffs[key] = cumulative_counts

# 選択回数の差分データフレーム
selection_diffs_df = pd.DataFrame(selection_diffs).fillna(0)
print("Selection Differences:")
print(selection_diffs_df)

# 累積選択人数の差分データフレーム
cumulative_diffs_df = pd.DataFrame(cumulative_diffs).fillna(0)
print("\nCumulative Selection Differences:")
print(cumulative_diffs_df)


In [ ]:
# CSVファイルに保存
output_filename = 'cumulative_diffs.csv'
cumulative_diffs_df.to_csv(output_filename, index=True)

print(f"Data saved to {output_filename}")


In [ ]:
import zipfile

# ZIPファイルの名前を指定
zip_filename = 'graphs.zip'

# 生成したグラフの画像をZIPファイルに圧縮
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk(output_dir):
        for file in files:
            if file.endswith(".png"):
                zipf.write(os.path.join(root, file), file)


In [ ]:
import numpy as np

def calculate_cumulative_tourists(simulation_results):
    total_tourists = 0
    for month_data in simulation_results.values():
        total_tourists += month_data.shape[0]
    return total_tourists

cumulative_tourists_list = []

# 10回のシミュレーションを実行
for _ in range(10):
    # 仮のシミュレーション実行関数（条件は適宜設定）
    simulation_results = run_simulation(predicted_counts_df, facilities, plans)
    cumulative_tourists = calculate_cumulative_tourists(simulation_results)
    cumulative_tourists_list.append(cumulative_tourists)

# 平均と標準偏差を計算
average_cumulative_tourists = np.mean(cumulative_tourists_list)
std_dev_cumulative_tourists = np.std(cumulative_tourists_list)

print(f"Average Cumulative Tourists: {average_cumulative_tourists}")
print(f"Standard Deviation: {std_dev_cumulative_tourists}")



##シナリオテスト